D1部分内容

In [ ]:
import tushare as ts 
ts.set_token('3d5a22139cda9c4f4e94ff39c1f26159a637b73d78ad26b1083c289c')

pro = ts.pro_api()
df = pro.query('daily', ts_code='600000.SH', start_date='20230101', end_date='20230110')
print(f"数据获取成功! 形状: {df.shape}") 
print(f"列名：{df.columns.tolist()}")
 # 获取所有曾上市的股票（含退市）
# all_stocks = pro.stock_basic(exchange='', list_status='D', fields='ts_code,symbol,name,area,industry,list_date,delist_date')
# print(f"历史股票总数: {len(all_stocks)} (含退市)")

# # 合并当前上市股票
# active_stocks = pro.stock_basic(exchange='', list_status='L')
# all_stocks = pd.concat([all_stocks, active_stocks]).drop_duplicates('ts_code')
# print(f"✅ 去重后总股票数: {len(all_stocks)}")
import pandas as pd
import time
import os
from datetime import datetime
from tqdm import tqdm
#=======================================
#获取全部股票并缓存，接口限制
#=======================================
os.makedirs('data_cache', exist_ok=True)
CACHE_FILE = 'data_cache/all_stocks_full.pkl'
def get_all_stocks_with_cache():
    """智能获取全部股票数据（含已退市），带本地缓存"""
    
    # 检查缓存是否存在且24小时内有效
    if os.path.exists(CACHE_FILE):
        file_time = datetime.fromtimestamp(os.path.getmtime(CACHE_FILE))
        if (datetime.now() - file_time).total_seconds() < 86400:  
            print(f"✅ 从本地缓存加载全量股票数据 (更新时间: {file_time.strftime('%Y-%m-%d %H:%M')})")
            return pd.read_pickle(CACHE_FILE)
    
    print("⏳ 正在从Tushare获取全量股票数据（含已退市）...")
    
    try:
        all_stocks = pro.stock_basic( exchange='', list_status='', 
            fields='ts_code,symbol,name,area,industry,list_status,list_date,delist_date'
        )
        
        # 保存到缓存
        all_stocks.to_pickle(CACHE_FILE)
        print(f"✅ 成功获取 {len(all_stocks)} 只股票数据（含已退市）")
        print(f"💾 数据已缓存至: {CACHE_FILE}")
        return all_stocks
        
    except Exception as e:
        print(f"❌ 获取数据失败: {str(e)}")
        
        # 尝试加载旧缓存（即使过期）
        if os.path.exists(CACHE_FILE):
            print("🔄 加载最近一次缓存数据（可能不是最新\)")
            return pd.read_pickle(CACHE_FILE)
        else:
            raise

all_stocks = get_all_stocks_with_cache()
# active_stocks = all_stocks.loc(all_stocks['list_status'] == 'L')
# delisted_stocks = all_stocks.loc(all_stocks['list_status'] == 'D')
# pause_stocks = all_stocks.loc(all_stocks['list_status'] == 'P')
#========================================
#批量下载日线数据
#==================================

# 按照日期快速加载及基础缓存
os.makedirs('data_cache/daily', exist_ok=True)


def get_all_daily_data(start_date,end_date,force_refresh=False):

    #缓存以及基础检查
    cache_file = f"data_cache/daily/{start_date}_{end_date}.pkl"
    # 强制刷新或清理无效缓存
    if force_refresh and os.path.exists(cache_file):
        print(f"🧹 清理旧缓存: {cache_file}")
        os.remove(cache_file)
    
    if os.path.exists(cache_file):
        print(f"从缓存加载 {start_date}~{end_date} 数据")
        return pd.read_pickle(cache_file)

    #tusahre建议，按照trade_date来提取
    trade_cal = pro.trade_cal(exchange='', start_date=start_date, end_date=end_date)
    trade_days = trade_cal[trade_cal.is_open == 1]['cal_date'].tolist()
    print(f"共 {len(trade_days)} 个交易日需要获取")
    
    all_data = []
    
    for i in tqdm(trade_days, desc="下载日线数据"):
           
        try:
            df = pro.daily(
                trade_date=i,
                fields=''
            )
            if not df.empty and 'ts_code' in df.columns:
                all_data.append(df)
            else:
                print(f"⚠️ 跳过无效批次: {i} "
                      f"(行数: {len(df)}, 列数: {len(df.columns)})")
            time.sleep(0.3)
        except Exception as e:
            print(f" 日期区间 {i} 获取失败: {str(e)}")
            time.sleep(3)
    
    if all_data:
        daily_df = pd.concat(all_data, ignore_index=True)
        if 'ts_code' not in daily_df.columns:
            print(f"⚠️ 数据缺少ts_code列! 返回空DataFrame. 实际列: {daily_df.columns.tolist()}")
        # 用category类型压缩内存 这一步很重要
        daily_df['ts_code'] = daily_df['ts_code'].astype('category')
        daily_df.to_pickle(cache_file)
        return daily_df
    return pd.DataFrame()


start_date = '20220101'
end_date = '20251201' 
daily_data = get_all_daily_data(start_date, end_date)
   


数据获取成功! 形状: (6, 11)
列名：['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close', 'change', 'pct_chg', 'vol', 'amount']
✅ 从本地缓存加载全量股票数据 (更新时间: 2025-12-23 17:05)
共 947 个交易日需要获取


下载日线数据: 100%|██████████| 947/947 [09:01<00:00,  1.75it/s]


数据处理部分，复权处理，ST股处理，缺失值异常值处理

In [ ]:
#================================================
#复权处理
#================================================
os.makedirs('data_cache/adj_factors', exist_ok=True)

if 'ts_code' in daily_data.columns and not daily_data.empty:
    print(f"✅ 获取{len(daily_data)}行日线数据，覆盖{daily_data['ts_code'].nunique()}只股票")
else:
    print("❌ 数据获取失败，请检查API响应")
print(daily_data.head())
def get_adj_factors(daily_df,start_date,end_date,force_refresh=False):
    
    cache_file = f"data_cache/adj_factors/{start_date}_{end_date}.pkl"
    # 强制刷新或清理无效缓存
    if force_refresh and os.path.exists(cache_file):
        print(f"🧹 清理旧缓存: {cache_file}")
        os.remove(cache_file)
    
    if os.path.exists(cache_file):
        print(f"从缓存加载 {start_date}~{end_date} 数据")
        return pd.read_pickle(cache_file)

    #tusahre建议，按照trade_date来提取
    trade_cal = pro.trade_cal(exchange='', start_date=start_date, end_date=end_date)
    trade_days = trade_cal[trade_cal.is_open == 1]['cal_date'].tolist()
    print(f"共 {len(trade_days)} 个交易日需要获取")
    # 获取复权因子（关键！）
    adj_factors = []
    for i in tqdm(trade_days, desc="获取复权因子"):
        try:
            df_adj = pro.adj_factor(ts_code='',trade_date=i)
            if not df_adj.empty and 'ts_code' in df_adj.columns:  
                adj_factors.append(df_adj)
            time.sleep(0.15)
        except Exception as e:
            print(f"日期 {i} 获取失败: {str(e)}")
            time.sleep(3)

    adj_df = pd.concat(adj_factors, ignore_index=True)

    # 合并复权因子
    daily_df = pd.merge(daily_df, adj_df[['ts_code','trade_date','adj_factor']], 
                        on=['ts_code','trade_date'], how='left')

    # 复权计算
    daily_df['open_adj'] = daily_df['open'] * daily_df['adj_factor']
    daily_df['high_adj'] = daily_df['high'] * daily_df['adj_factor']
    daily_df['low_adj'] = daily_df['low'] * daily_df['adj_factor']
    daily_df['close_adj'] = daily_df['close'] * daily_df['adj_factor']

    #释放内存一下
    daily_df=daily_df.copy()
    daily_df.to_pickle(cache_file)
    return daily_df

adjdaily_data = get_adj_factors(daily_data,start_date,end_date)
print(adjdaily_data.head())

✅ 获取4925784行日线数据，覆盖5621只股票
     ts_code trade_date   open   high    low  close  pre_close  change  \
0  000001.SZ   20251201  11.60  11.70  11.53  11.69      11.61    0.08   
1  000002.SZ   20251201   5.30   5.34   5.17   5.20       5.38   -0.18   
2  000004.SZ   20251201  11.93  12.36  11.78  12.36      11.77    0.59   
3  000006.SZ   20251201  10.88  10.91  10.35  10.43      11.01   -0.58   
4  000007.SZ   20251201   9.91  10.87   9.88  10.87       9.88    0.99   

   pct_chg         vol       amount  
0   0.6891  1037322.71  1205480.323  
1  -3.3457  3212116.25  1684908.905  
2   5.0127    92745.00   113410.422  
3  -5.2679   421976.64   444839.931  
4  10.0202   167432.47   175964.693  
🧹 清理旧缓存: data_cache/adj_factors/20220101_20251201.pkl
共 947 个交易日需要获取


获取复权因子: 100%|██████████| 947/947 [05:16<00:00,  2.99it/s]


     ts_code trade_date   open   high    low  close  pre_close  change  \
0  000001.SZ   20251201  11.60  11.70  11.53  11.69      11.61    0.08   
1  000002.SZ   20251201   5.30   5.34   5.17   5.20       5.38   -0.18   
2  000004.SZ   20251201  11.93  12.36  11.78  12.36      11.77    0.59   
3  000006.SZ   20251201  10.88  10.91  10.35  10.43      11.01   -0.58   
4  000007.SZ   20251201   9.91  10.87   9.88  10.87       9.88    0.99   

   pct_chg         vol       amount  adj_factor    open_adj    high_adj  \
0   0.6891  1037322.71  1205480.323    134.5794  1561.12104  1574.57898   
1  -3.3457  3212116.25  1684908.905    181.7040   963.03120   970.29936   
2   5.0127    92745.00   113410.422      4.0640    48.48352    50.23104   
3  -5.2679   421976.64   444839.931     39.7400   432.37120   433.56340   
4  10.0202   167432.47   175964.693      8.2840    82.09444    90.04708   

       low_adj    close_adj  
0  1551.700482  1573.233186  
1   939.409680   944.860800  
2    47.873920

In [4]:
print(adjdaily_data.loc[(adjdaily_data['trade_date']=='20251127')&(adjdaily_data['ts_code']=='920985.BJ')])

         ts_code trade_date  open  high   low  close  pre_close  change  \
16297  920985.BJ   20251127  8.65  8.77  8.58   8.58       8.69   -0.11   

       pct_chg       vol     amount  adj_factor  open_adj  high_adj   low_adj  \
16297  -1.2658  22831.79  19803.047       1.628   14.0822  14.27756  13.96824   

       close_adj  
16297   13.96824  


In [ ]:
#===========================
#数据清洗（ST、退市、缺失值、异常值）
#===========================

def clean_stock_data(daily_df: pd.DataFrame, all_stocks: pd.DataFrame) -> pd.DataFrame:
    """
    专业数据清洗流水线
    :param daily_df: 原始日线数据
    :param all_stocks: 包含退市日期的股票元数据
    :return: 清洗后的DataFrame
    """
    # 1. 创建工作副本（保护原始数据）
    df = daily_df.copy()
    all_stocks['delist_date'] = pd.to_datetime(all_stocks['delist_date'], errors='coerce')
    # 2. ST标记（安全处理*ST）
    df['is_st'] = df['name'].str.contains(r'\*?ST', regex=True)
    
    # 3. 智能退市过滤
    #   - 合并退市日期（保留所有股票）
    df = pd.merge(
        df, 
        all_stocks[['ts_code', 'delist_date']].drop_duplicates(),
        on='ts_code', how='left'
    )
    
    #   - 转换日期类型
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df['delist_date'] = pd.to_datetime(df['delist_date'])
    
    #   - 创建安全掩码（核心逻辑）
    mask = (
        # 未退市股票：无退市日期
        df['delist_date'].isna() |
        # 已退市股票：在退市日+30天内
        (df['trade_date'] <= df['delist_date'] + pd.Timedelta(days=30))
    )
    df = df[mask]
    
    # 4. 创建清洗集（非ST + 保留有效退市数据）
    clean_df = df[~df['is_st']].copy()  # 只过滤ST，保留有效退市数据
    
    # 5. 严格时间排序（收益率计算前提）
    clean_df = clean_df.sort_values(['ts_code', 'trade_date']).reset_index(drop=True)
    
    # 6. 收益率计算（在clean_df上！）
    clean_df['ret'] = clean_df.groupby('ts_code')['close_adj'].pct_change()
    
    # 7. 安全极端值过滤（保留首日）
    clean_df = clean_df[
        (clean_df['ret'].isna()) |  # 保留首日
        (abs(clean_df['ret']) < 0.2)
    ]
    
    # 8. 智能缺失值处理
    #   - 价格：前向填充（停牌时用前值）
    clean_df['open_adj'] = clean_df.groupby('ts_code')['open_adj'].ffill()
    clean_df['close_adj'] = clean_df.groupby('ts_code')['close_adj'].ffill()
    
    #   - 成交量：停牌时填0
    clean_df['vol'] = clean_df.groupby('ts_code')['vol'].fillna(0)
    
    # 9. 日期标准化（防御性转换）
    if not pd.api.types.is_datetime64_any_dtype(clean_df['trade_date']):
        clean_df['trade_date'] = pd.to_datetime(clean_df['trade_date'])
    clean_df['trade_date'] = clean_df['trade_date'].dt.strftime('%Y%m%d')
    
    # 10. 内存优化
    clean_df['ts_code'] = clean_df['ts_code'].astype('category')
    
    return clean_df


clean_data = clean_stock_data(adjdaily_data, all_stocks)  

In [ ]:
#==================================
#D2交付与验证
#==================================
# 检查停牌股票（连续3天成交量=0）
suspend = clean_data[clean_data['vol']==0].groupby('ts_code').filter(lambda x: len(x)>3)
print(f"停牌超3天股数: {suspend['ts_code'].nunique()}")

clean_data.to_hdf('/content/drive/MyDrive/cleaned_daily.h5', key='daily', mode='w')

# 必须通过的检查
assert clean_data.isnull().sum().max() == 0, "存在未处理缺失值!"
assert clean_data[clean_data['close_adj'] < 0].empty, "出现负价格!"
print("✅ D2数据清洗验证通过! 保存至cleaned_daily.h5")

D3 因子基础数据生成：获取财务数据，生成基础因子（含行业对齐），构建面板数据集（最终交付）

In [ ]:
#=================================
#获取财务数据（我的积分！！！！！）
#=================================
def get_financial_data(df):
    financial_cols = ['ts_code', 'ann_date', 'end_date', 'roe', 'debt_to_assets', 'total_mv']
    financial_data = []

    # 按股票分批下载（降低请求量）
    for ts_code in tqdm(df['ts_code'].unique()[:200], desc="下载财务数据"):  # 先试200只
        try:
            df_fin = pro.fina_indicator(ts_code=ts_code, 
                                    start_date=start_date, 
                                    end_date=end_date,
                                    fields=','.join(financial_cols))
            if not df_fin.empty:
                financial_data.append(df_fin)
            time.sleep(0.3)  # 财务数据接口更敏感
        except Exception as e:
            print(f"财务数据失败 {ts_code}: {str(e)}")

    fin_df = pd.concat(financial_data, ignore_index=True)
    return fin_df

#========================================
#生成基础因子（行业对齐）
#========================================
def get_basic_factors(clean_df,fin_df):
    # 1. 计算基础收益
    clean_df['ret_next'] = clean_df.groupby('ts_code')['close_adj'].pct_change(-1)  # 次日收益

    # 2. 合并财务数据（关键：对齐公告日）
    fin_df['ann_date'] = pd.to_datetime(fin_df['ann_date'])
    clean_df['trade_date'] = pd.to_datetime(clean_df['trade_date'])
    merged_df = pd.merge_asof(clean_df.sort_values('trade_date'), 
                            fin_df.sort_values('ann_date'),
                            by='ts_code',
                            left_on='trade_date',
                            right_on='ann_date',
                            direction='backward')  # 取最近公告

    # 3. 生成经典因子
    merged_df['bp'] = 1 / merged_df['pb']  # 价值因子
    merged_df['mom_20'] = merged_df.groupby('ts_code')['close_adj'].pct_change(20)  # 动量
    merged_df['turnover'] = merged_df['vol'] / merged_df['total_share']  # 换手率

    #可以做一些因子处理比如去极值（winsorize）
    """ for col in ['bp', 'mom_20']:
            p1 = merged_df[col].quantile(0.01)
            p99 = merged_df[col].quantile(0.99)
            merged_df[col] = merged_df[col].clip(p1, p99) """
    # 4. 行业数据（用申万一级）
    industry = pro.index_member(index_code='399300.SZ')  # 沪深300成分
    merged_df = pd.merge(merged_df, industry[['ts_code','in_date','out_date','con_code']], 
                        on='ts_code', how='left')
    
    return merged_df

#============================
#构建面板数据集与最终验证
#=============================
final_cols = ['ts_code','trade_date','open_adj','high_adj','low_adj','close_adj','vol',
              'ret_next','bp','mom_20','turnover','industry']
panel_df = merged_df[final_cols].copy()

# 重置索引（符合alphalens要求）
panel_df = panel_df.set_index(['trade_date', 'ts_code']).sort_index()

# 保存最终数据集
panel_df.to_hdf('/content/drive/MyDrive/factor_panel.h5', key='panel', mode='w')
print(f"✅ 最终数据集: {panel_df.shape} (行,列)")

# 验证1：无未来函数
sample_stock = panel_df.xs('600000.SH', level=1).head(5)
print("前5行数据预览:")
print(sample_stock[['close_adj','bp','mom_20']].head())

# 验证2：因子分布
import matplotlib.pyplot as plt
panel_df['bp'].groupby('trade_date').mean().plot(title='平均BP时序图')
plt.savefig('/content/drive/MyDrive/bp_trend.png')

# 验证3：缺失率检查
missing_rate = panel_df.isnull().mean() * 100
print(f"关键字段缺失率:\n{missing_rate[['bp','mom_20']]}")
# 合格标准: <5%（财务数据天然缺失合理）